In [ ]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import SimpleSequentialChain, LLMChain,TransformChain
from dotenv import load_dotenv
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.agents import load_tools,initialize_agent,AgentType,create_react_agent,AgentExecutor
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.docstore.document import Document
from langchain_core.prompts import PromptTemplate
import streamlit as st

#### Creacion del modelo 

In [4]:
load_dotenv()
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature= 0.2)
embedings = OpenAIEmbeddings()

#### Vectorizacion del pdf ####

 

In [5]:
def vectorizacion_pdf(pdf):
    loader = PDFPlumberLoader(pdf)
    docs = loader.load()
    text = "\n".join([page.page_content for page in docs])
    pdf_splitter = CharacterTextSplitter(chunk_size=900, chunk_overlap=80)
    texts = pdf_splitter.split_text(text)
    documents = [Document(page_content=t) for t in texts]
    persist_path=f"./{pdf}_bd"  #ruta donde se guardará la BBDD vectorizado
    #Creamos la BBDD de vectores a partir de los documentos y la función embeddings
    vector_store = SKLearnVectorStore.from_documents(
        documents=documents,
        embedding=embedings,
        persist_path=persist_path,
        serializer="parquet", #el serializador o formato de la BD lo definimos como parquet
    )
    return vector_store.persist()

#### Consulta sobre el pdf ####

In [6]:
def consulta_pdf(vector_store,consulta):
    #Cargamos la BBDD de vectores
    vector_store_connection = SKLearnVectorStore.load(vector_store)
    if vector_store_connection is None:
        print("No se ha encontrado una instancia de la BBDD de vectores en ", vector_store)
        return None
    else:
        print("Una instancia de la BBDD de vectores se ha cargado desde ", vector_store)
    #Realizamos la consulta a la BBDD de vectores
        docs = vector_store_connection.similarity_search(consulta, k = 2)
        print(docs[0].page_content)
        return docs

#### consulta sobre el extracto de pdf ####

In [31]:
prompt_template = PromptTemplate.from_template(
    "Eres un agente que responde preguntas {input} sobre un extracto de PDF {extracto}"
)

In [33]:
qa_chain = LLMChain(llm=llm, prompt=prompt_template)

#### FRONT

In [ ]:
st.title("Consulta de documentos PDF")
st.write("Este programa permite realizar consultas en documentos PDF...solo pdf por ahora")





2025-02-15 17:49:26.598 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 17:49:26.944 
  command:

    streamlit run c:\Users\tomas\anaconda3\envs\langchain\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-02-15 17:49:26.945 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()